In [18]:
#import packages
import torch
import numpy as np
import scipy as sp
from scipy import sparse
from scipy.sparse import dia_matrix
import scqubits as sc
from zeropi import ZeroPi
from typing import Any, Callable, Dict, List, Optional, Tuple, Union, cast
import noise



In [19]:
#scqubit fixed params
omega_low = 1e-9 * 2 * np.pi  # Low frequency cutoff. Units: 2pi GHz
omega_high = 3 * 2 * np.pi  # High frequency cutoff. Units: 2pi GHz
t_exp = 1e4  #Measurement time. Units: ns
A_cc = 1e-7 # Critical current noise strength. Units of critical current I_c
A_flux = 1e-6   # Flux noise strength. Units: Phi_0
M= 400  # Mutual inductance between qubit and a flux line. Units: \Phi_0 / Ampere
R_0 = 50 # Characteristic impedance of a transmission line. Units: ohms
T = 0.015 # Typical temperature for a superconducting circuit experiment. Units: K

In [20]:
#Computing T2
EJ = 0.4
EL = 10.0**(-2)
ECJ = 0.5
EC = 1
ECS = 10.0**(-3)
dEJ = 0
dCJ = 0
ng = 0.1
flux = 0.23
ncut = 30
truncated_dim = 10
pt_count = 10
min_val = -6*np.pi
max_val = 6*np.pi


qubit = ZeroPi(EJ, EL, ECJ, EC, dEJ, dCJ, ng, flux, ncut, truncated_dim, pt_count, min_val, max_val)

In [21]:
#Solve H
eigvals,eigvecs = qubit.esys()
omega = 2 * np.pi * (eigvals[1]-eigvals[0]) * 1e9

#Tphi 
tphi_1_over_f_cc = noise.tphi(
    A_noise = A_cc, 
    noise_op= qubit.d_hamiltonian_d_EJ(),
    eigvals=eigvals, eigvecs=eigvecs, 
    omega_low = omega_low, 
    t_exp = t_exp
    )

tphi_1_over_f_flux=noise.tphi(
    A_noise = A_flux, 
    noise_op= qubit.d_hamiltonian_d_flux(), 
    eigvals=eigvals, eigvecs=eigvecs,
    omega_low = omega_low, 
    t_exp = t_exp
    )

#T1
t1_inductive = noise.t1(
    noise_op=qubit.phi_operator(), 
    spectral_density=ZeroPi.spectral_density_ind(qubit, T=T), 
    eigvals=eigvals, eigvecs=eigvecs, EL = EL)

t1_flux_bias_line = noise.t1(
    noise_op=qubit.d_hamiltonian_d_flux(), 
    spectral_density=ZeroPi.spectral_density_fbl(qubit, M=M, R_0 = R_0, T=T), 
    eigvals=eigvals, eigvecs=eigvecs,  EL =None)

#t2
total_t1_rate = t1_inductive + t1_flux_bias_line
total_tphi_rate = tphi_1_over_f_flux+tphi_1_over_f_cc
total_t2_rate = total_t1_rate + 0.5*total_tphi_rate
total_t2 = 1/total_t2_rate

total_t2.item()

504047.8312891457

In [22]:
#compare my pytorch computations to scqubits
phi_grid = sc.Grid1d(min_val=min_val, max_val=max_val, pt_count=pt_count)
zeropi = sc.ZeroPi(
            grid = phi_grid,
            EJ   = EJ,
            EL   = EL, 
            ECJ  = ECJ,
            EC   = EC,
            ng   = ng,
            flux = flux,
            ncut = 30, 
            dEJ = dEJ, 
            dCJ = dCJ)

print(zeropi.t1_inductive() - 1/t1_inductive.item())
print(zeropi.t1_flux_bias_line() - 1/t1_flux_bias_line.item())
print(zeropi.tphi_1_over_f_cc() - 1/tphi_1_over_f_cc.item())
print(zeropi.tphi_1_over_f_flux() - 1/tphi_1_over_f_flux.item())

-1000764728.5215592
-3.528899117520406e+20
-3.474997356534004e-08
1.3747485354542732e-06
